# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [1]:
import numpy as np
import pandas as pd
import pickle
import math
from nltk.tokenize import sent_tokenize


from IPython.display import Markdown, display

In [2]:
import sys
sys.path.append('..')

In [3]:
# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', -1)

# %autosave 50

In [4]:
# import gzip
# import math
# import random
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.ensemble import GradientBoostingRegressor, \
# GradientBoostingClassifier, RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import SGDClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix, \
# classification_report, make_scorer
# import statsmodels.api as sm

In [5]:
# From this project
# from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse
from NLP import WordBag, AboutMovie

In [6]:
%reload_ext autoreload
from subjective_filter import SubjectiveFilter

## Configuration

In [7]:
# Subsampling from Amazon reviews
NB_SAMPLES = 360000 #4000  # up to 200k, then change the input file

data_path = '../../../datasets/'


In [78]:
dd = pd.DataFrame({'A': [1,2], 'B':[1,3]})
df.reset_index().reset_index().rename(columns={'level_0': 'asin'})

,asin,index,reviewerID,asin,reviewText,overall
0,0,861752,A175U4QHQUXC69,6303625800,The DVD would not play anything but the selection page for color or black and white version. Only the introduction music plays,1.0
1,1,861865,A46HSTZBUZ5ZH,6303625800,"Being a definitive fan of Laurel and Hardy, and having a special place in my heart for ""Babes In Toyland"" (aka ""March of the Wooden Soldiers""), I was excited to add another version to my DVD collection.Alas, the colorization from Legend Films is a big disappointment. Already in possession of the excellent 2001 Goodtimes Home Video release, in which the colorization truly does approximate the technicolor look, I feel as though I have wasted my money. As for the black and white print, also included on the DVD, is a much poorer rendering than the pristine black and white version available through MGM Home Video.The extras on the Legends release are nothing new. They have been included on other L&H; releases by other companies, so there is nothing gained there.Over all, I would suggest that if your heart is set on having a colorized version of this particular feature, stick with the 2001 Goodtimes version. The effect is much more pleasing to those of us wanting a ""technicolor"" appearence. As for the black and white version, stick with MGM's release. It truly is the best print available.Sorry, Legend Films. You've missed the mark on this one!",1.0
2,2,861912,A3KEABH9BZ1MPB,6303625800,The adults found it very boring and the children ages 8 and 10 said it was just ok,1.0
3,3,861931,A1EBEAG4PJOIB2,6303625800,"Love this film, especially at Thanksgiving. But as long as it's only available in the phony and insulting colorized version I won't touch it. This is a piece of art. Shame on those who tamper with this and other films. It's like someone drawing a mustache on the Mona Lisa.",1.0
4,4,862191,AR3UW0VBY55M8,6303625800,I purchased this in error. I really wanted the DVD which has both the black and white and the colorized versions.,1.0
5,5,862419,A3MV1KKHX51FYT,6303626475,"This is the film version of the J.B. Priestly horror novel. It was a dark and stormy night, a couple is driving on a muddy country road through the Welsh mountains. Their touring car does not have side curtains. They are lost, and pull into a house with lights. The Femm household has some strange characters. This house has its own electricity. There is conflict among the household, like when prayers are offered before a meal. Then a knock on the door brings another couple to this house; they are also lost. The guests tell about themselves. [Filming in the rain must have been a technical breakthrough that year.]When they go to fetch a lamp they hear a strange sound. Is there someone in a locked room? Does the butler have a drinking problem? Will something terrible happen? If a madman is locked up what will happen if he is released? Will the innocent survive and the guilty die?This is boring and almost unwatchable film, a waste of time. It does show the performances of some actors who became more famous years later. Some of the scenes reflect the hidden quirks of the actors.",1.0
6,6,862466,AM55WQ075SIK7,6303626475,the movie called The Old Dark House should be called The Old Dark Horrible Movie. Terrible!!!!! I'm sorry I listened to the other reviews. I gave it to Goodwill.,1.0
7,7,862856,A9XKE4OE48BNK,6303631940,"This is actually neck-in-neck with Colin Baker's horrible regeneration story 'The Twin Dilemma'. This mess isn't really Sylvester McCoy's fault. The script is wretched, the plot is nonsensical (even by Who standards), and the Rani actually seems to be looking at the TV audience at one point. Ugh. It, like all of McCoy's first season, also commits the cardinal sin of being both excruciating and boring. Don't give up on McCoy though. Check out some of his stories from later in his run like Remembrance of the Daleks and Curse of Fenric.",1.0
8,8,862861,AKJOWY6V1EJPJ,6303631940,"As with the sadly under-rated Co

In [8]:
# file_name = '360000_balanced_train_test_reviews.pkl'
file_name = '_balanced_pos_neg_train_test_reviews.pkl'

pickle_in = open(data_path + str(NB_SAMPLES) + file_name,"rb")
train_test_dic0 = pickle.load(pickle_in)

In [79]:
SAMPLE_FRACTION = 0.04

test_dic = {'train': {}, 'test':{}}

for i in ['train','test']:
    for j in ['positive','negative']:
         test_dic[i][j] = train_test_dic0[i][j] \
            .iloc[:math.floor(len(train_test_dic0[i][j].index) * SAMPLE_FRACTION), :] \
            .reset_index() \
            .reset_index() \
            .drop(['reviewerName', 'helpful', 'asin', 'index',
                   'summary', 'unixReviewTime', 'reviewTime'], axis=1) \
            .rename(columns={'level_0': 'asin'})


In [80]:
test_dic['train']['positive']

,asin,reviewerID,reviewText,overall
0,0,A32244V7CQUBD6,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love. It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0
1,1,A32244V7CQUBD6,"This episode pretty much has Hafron and Emmanuelle teleporting to different parts of the world and &quot;doing it&quot;. There is the continuing plot from an earlier episode of some group on Earth trying to track them down. That's the main reason for Emmanuelle and Hafron to jump to different parts of the world. Otherwise, this episode is mostly sex scenes.",4.0
2,2,A33KKMGGVLZ29T,"This is an intimate concert of Robert Mirabal. Although I thought that it was, as I said, masterful, the sound, at times sounded a little muffled.The storytelling of the songs gave an insight of native culture and of Mirabal's own family stories and history.The Dance and Ee You Oo are my picks for the best songs, but they are all a joy to watch. The Rare Tribal Mob and the Mirabal Singers/Dancers are great and provide a mesmerising stage performance.Very enjoyable",4.0
3,3,A33P47VEH0YULL,"A well put together DVD for the ""Stinkiest band"" in the world. Easy to navigate, and with some pretty interesting side notes...a great collectors item for any fan of Cradle of Filth or Swedish metal.",4.0
4,4,ADG33WELAQRZJ,"Great show from the inventors of the extreme gothic metal genre !! 75'of pure mayhem with good visuals and a great sound, although the vocals are a little high in the mix for my taste and the guitar on the right channel is 10 times louder than the one on the left so I ended up switching it to mono to even things up. It's hard to give this more stars esp. when you've watched the PTSFirepower DVD first like I did, that one is so much better. The 5 bonus videoclips totally rule and round up the package making the purchase a worthy one.Now the mockumentary may not be suited to everyone's tastes, personally it bored me and know of a lot of people who feels it ruins the band's image. Well don't watch it then.",4.0
5,5,A3YXITJWFW4BW,"I rented this dvd, i didnt buy it. I also rented PanDamonAeon and Mannequin. This dvd has excellent very long concert footage. Its practically the same songs as on Livebait for the Dead. And then we have the rest of the dvd. Im sorry, i love CoF but that ""schockumentary"" thing was just sad. I could barely stand to watch it. It's stupid and boring, maybe if theyd shown more of the band in a more coherent manner...and it just goes on and on forever. This is a 2 hour plus dvd adn half of it is crap. The ""blair twit Project"" is slightly amusing, but mostly boring. The music videos are good though. I would still recommend buying this dvd just for the music videos and the concert footage. PanDamonAeon gave me more joy though.",4.0
6,6,A214NHULS3H7OX,"Excellent Sound, Excellent Picture, Tons and Tons of songs. Love the Scorched Earth Erotic video. The only reason I didn't give this 5 stars is because I don't think they let their female singer sing enough, it adds a nice goth touch. I also did not give it five stars because it would have been nice to have the ""her ghost in the fog"" video on here, and finally I thin Dani could have done better on some parts of the songs. For example: On ""her ghost in the fog"" (live version), he chooses to growl in that typical death metal voice, instead of talking in the deep gothic voice like he does on the album. By using this growly voice, I think he is cheese'n off some of the songs best parts. Anyway, these are minor gripes and this is a must have DVD, so definitely BUY IT!...",4.0
7,7,A2IZOU2G1QX0JD,"As most of my friends hate, I'm massively into Cradle of Filth. In the style of Napoleon Dynamite describing a Liger, I would say ""...they're pretty much my favorite band."" Their music is among the worst in the industry, they suck in a 

In [60]:
total = 0
for i in ['train','test']:
    for j in ['positive','negative']:
        print(test_dic[i][j].shape)
        total += test_dic[i][j].shape[0]
total

(5760, 4)
(5760, 4)
(1440, 4)
(1440, 4)


14400

In [61]:
# pickle_in = open(data_path
#                 + 'movie_reviews_144000Pos_Neg_Samples.pkl'
#                 , "rb")
# test_dic = pickle.load(pickle_in)

## Remove objective sentences for case B

In [62]:
%reload_ext autoreload
obj_path = '../obj_subj_dev/'
fit_obj_tf = obj_path + 'fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl'
fit_obj_model = obj_path + 'GBC_300_0.5_5_0.88cv.pkl'
subj_filter = SubjectiveFilter(fit_obj_tf, fit_obj_model)

In [63]:
# for tt in ['train','test']:
#     for pn in ['positive','negative']:
#         print(tt,pn,test_dic[tt][np].shape)
total = 0
for tt in test_dic.values():
    for df in tt.values():
        display(df.head(1))
        total += df.shape[0]
        display(df.shape)
print(total)

,reviewerID,asin,reviewText,overall
1515619,A32244V7CQUBD6,B00005QFEK,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love. It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0


(5760, 4)

,reviewerID,asin,reviewText,overall
861729,A3CFT5LHFCB46K,630362572X,"In the book The Official Godzilla Compendium, King Kong Vs Godzilla is described as ""The Jaws of the Japanese film Industry"" or ""One of the great monster battles of cinema history"". It is referred to as the film that ""...Lifted Godzilla from the swelling ranks of interchangeable atomic monsters of the fifties and placed him among the pantheon of cinema creatures"". Thus, I had high expectations when I watched this on the Sci-Fi channel.All I have to say is: Wow! The special effects are incredible! The acting is top notch. The screenplay is well written. This film is truly a classic of cinema that will leave you thinking long after you leave the theater.Wait. That's my review for 2001: A Space Odyssey.Here's what I really think of King Kong vs. Godzilla.A thousand words cannot describe my dislike of the film. I know it's a Godzilla film, but did it need to be this bad? It starts off good, with an excellent piece of music at the beginning credits. But soon goes downhill as we are introduced to U.N Reporter Eric Carter. All he was put in for was to make the film longer and annoy us. He ""communicates"" with people in Japan via a satellite that looks more like a first grade science project than visual effects. He constantly tells us that ""Godzilla is heading for Japan"" or ""King Kong and Godzilla will battle."" Duh! What do you think King Kong and Godzilla will do? Have tea and crumpets?Speaking of Kong, I have seen costumes that look better than this Kong suit. I would also rather listen to Celine Dion and Elton John do a music number together than listen to Kong roar. He doesn't even open his mouth! Godzilla appears just fine, except for the fact he's afraid of thermal energy (Didn't he tear down an electrical fence in 1954 with ease?). Kong however gets power from the thermal energy. Gee, I would assume that all his fur would make him the perfect conductor of elctricity. Also, doesn't anyone else notice that the islands of the first Godzilla movies (The '50s to the '70s) always look the same? The main battles themselves are also letdowns.It was a grand idea that Toho had: Pitting two titans of cinema against each other. Too bad the Japanese did not use stop motion animation. Or make a Kong/Godzilla rematch. Or make better costumes or etc, etc. For a true classic of monster cinema, King Kong, the original Godzilla or Godzilla 1985 are better choices than this pile of ... If this was the film that made Godzilla famous, he should have been assassinated.Indeed this film will leave you thinking after you leave the theater: Why did I waste my time on this?(Note: Perhaps my criticism was a bit harsh. After all, this is the American version of the movie and we always [mess] with perfection. Maybe the Japanese version is better. Get that one if you can).",1.0


(5760, 4)

,reviewerID,asin,reviewText,overall
1515665,A2D832OA6Q5ZAS,B00005QFFP,"What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known. Like a great actress, she dissolves into her role, and yet her voice and style are easily recognizable for the effortless simplicity with which she nails every note and figure. I wish they could have taped in color in those days, and I would have liked to see her perform more examples of the coloratura repertoire and less of the popular romantic themes. Overall, I am delighted with this video.",4.0


(1440, 4)

,reviewerID,asin,reviewText,overall
861752,A175U4QHQUXC69,6303625800,The DVD would not play anything but the selection page for color or black and white version. Only the introduction music plays,1.0


(1440, 4)

14400


In [64]:
%reload_ext autoreload
from subjective_filter import SubjectiveFilter

REMOVE = 'obj'
REMOVE_FRACTION = 1/3
CHUNK_SZ = 1

sent_dfs = {'train':{},'test':{}}
nb_sentences_removed = 0

for ttname, tt in test_dic.items():
    for pn, df in tt.items():
        df_list = []
        start = 0
        while start < df.shape[0]:
            end = start + CHUNK_SZ
            df1 = df.iloc[start:end,:]
            df2 = subj_filter.to_one_sent_per_row(df1)
            df3, removed = subj_filter.transform(
                    df2,
                    'sentence', 
                    remove_fraction = REMOVE_FRACTION,
                    debug_level=0,
                    remove=REMOVE)
            if removed == -1:
                start = end
                continue
            df_list.append(df3)
            nb_sentences_removed += removed
            start = end

        if len(df_list) == 0:
            sent_dfs[ttname][pn] = None
            print('No reviews for {} {}'.format(ttname, pn))
            continue
        sent_dfs[ttname][pn] = df_list.pop()
        while len(df_list) > 0:
            sent_dfs[ttname][pn] = pd.merge(df_list.pop(), 
                                            sent_dfs[ttname][pn], how='outer')
#         res[ttname][pn] = reduce(lambda x, y: pd.merge(x, y, on = ['reviewerID','asin']), dfList)


In [65]:
display(Markdown('### {} sentences removed'.format(nb_sentences_removed)))

### 33808 sentences removed

In [66]:
df1 = sent_dfs['train']['positive']
df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]

,reviewerID,asin,sentence,overall
0,A32244V7CQUBD6,B00005QFEK,It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0


In [67]:
for ttname, tt in sent_dfs.items():
    for pn, df in tt.items():
        display(df.head(3))

,reviewerID,asin,sentence,overall
0,A32244V7CQUBD6,B00005QFEK,It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0
1,A32244V7CQUBD6,B00005QFEN,"This episode pretty much has Hafron and Emmanuelle teleporting to different parts of the world and &quot;doing it&quot;. Otherwise, this episode is mostly sex scenes. That's the main reason for Emmanuelle and Hafron to jump to different parts of the world.",4.0
2,A33KKMGGVLZ29T,B00005QFER,The Rare Tribal Mob and the Mirabal Singers/Dancers are great and provide a mesmerising stage performance.Very enjoyable This is an intimate concert of Robert Mirabal.,4.0


,reviewerID,asin,sentence,overall
0,A3CFT5LHFCB46K,630362572X,"I know it's a Godzilla film, but did it need to be this bad? If this was the film that made Godzilla famous, he should have been assassinated.Indeed this film will leave you thinking after you leave the theater: Why did I waste my time on this? Have tea and crumpets?Speaking of Kong, I have seen costumes that look better than this Kong suit. This film is truly a classic of cinema that will leave you thinking long after you leave the theater.Wait. For a true classic of monster cinema, King Kong, the original Godzilla or Godzilla 1985 are better choices than this pile of ... Get that one if you can). That's my review for 2001: A Space Odyssey.Here's what I really think of King Kong vs. Godzilla.A thousand words cannot describe my dislike of the film. It starts off good, with an excellent piece of music at the beginning credits. He ""communicates"" with people in Japan via a satellite that looks more like a first grade science project than visual effects. He constantly tells us that ""Godzilla is heading for Japan"" or ""King Kong and Godzilla will battle."" After all, this is the American version of the movie and we always [mess] with perfection. It is referred to as the film that ""...Lifted Godzilla from the swelling ranks of interchangeable atomic monsters of the fifties and placed him among the pantheon of cinema creatures"". Or make better costumes or etc, etc. Too bad the Japanese did not use stop motion animation. In the book The Official Godzilla Compendium, King Kong Vs Godzilla is described as ""The Jaws of the Japanese film Industry"" or ""One of the great monster battles of cinema history"". (Note: Perhaps my criticism was a bit harsh. The main battles themselves are also letdowns.It was a grand idea that Toho had: Pitting two titans of cinema against each other. Also, doesn't anyone else notice that the islands of the first Godzilla movies (The '50s to the '70s) always look the same? The acting is top notch. The special effects are incredible! I would also rather listen to Celine Dion and Elton John do a music number together than listen to Kong roar. Or make a Kong/Godzilla rematch.",1.0
1,A1O2XS7W1TUI13,630362572X,"This isn't the '80's any more, is it? Boycott rip-offs like this! Why would someone buy this pan & scan trash?",1.0
2,A2UMZUR8VFEOYB,6303625800,"I wanted to watch with my grandson, but was so disappointed.Story line and content at times was terrible.",1.0


,reviewerID,asin,sentence,overall
0,A2D832OA6Q5ZAS,B00005QFFP,"Overall, I am delighted with this video. Like a great actress, she dissolves into her role, and yet her voice and style are easily recognizable for the effortless simplicity with which she nails every note and figure. I wish they could have taped in color in those days, and I would have liked to see her perform more examples of the coloratura repertoire and less of the popular romantic themes.",4.0
1,A1JF78EP4GPAOO,B00005QG2N,I would have liked this better if it included the Pandaemonaeon music videos and Her Ghost in the Fog. Where was the BBC documentary? The rest however seems to be uninspired filler.,4.0
2,AQ7K219573Z8P,B00005QIVF,"This is well worth watching, and is a good, rare Hong Kong drama. So this movie will not appeal to many western tastes. The contrast really is not believable, so it does detract some from the film. This is kind of a typical story about a very rough guy who has buried very deep a somewhat heart of gold. But this is made up for by Lau Chin Wan's supurb acting, and by Johnny To's tight directing.",4.0


,reviewerID,asin,sentence,overall
0,A175U4QHQUXC69,6303625800,The DVD would not play anything but the selection page for color or black and white version.,1.0
1,A46HSTZBUZ5ZH,6303625800,"It truly is the best print available.Sorry, Legend Films. You've missed the mark on this one! The effect is much more pleasing to those of us wanting a ""technicolor"" appearence. Already in possession of the excellent 2001 Goodtimes Home Video release, in which the colorization truly does approximate the technicolor look, I feel as though I have wasted my money. As for the black and white version, stick with MGM's release.",1.0
2,A1EBEAG4PJOIB2,6303625800,But as long as it's only available in the phony and insulting colorized version I won't touch it. It's like someone drawing a mustache on the Mona Lisa. This is a piece of art.,1.0


In [68]:
# df1 = sent_dfs['train']['positive']
# df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]
# df1

In [69]:
subj_dfs = sent_dfs

In [70]:
total = 0
old_total = 0
for tt in ['train', 'test']:
    for pn in ['positive', 'negative']:
        print('\n {} {}'.format(tt,pn))
        tot = test_dic[tt][pn].shape[0]
        new = subj_dfs[tt][pn].shape[0]
        dropped = tot - new
        print('Dropped: {0} ({1:.1%})'.format(dropped, dropped/tot))
        total += new
        old_total += tot
print('\nOld total:', old_total)
print('New total:', total)
print('Removed a total of:', old_total - total, 'reviews')


 train positive
Dropped: 367 (6.4%)

 train negative
Dropped: 442 (7.7%)

 test positive
Dropped: 105 (7.3%)

 test negative
Dropped: 132 (9.2%)

Old total: 14400
New total: 13354
Removed a total of: 1046 reviews


In [72]:
pickle_out = open(data_path
                    + 'reviews_wout_top_' + str(int(round(REMOVE_FRACTION * 100))) + 'pct_' + REMOVE + '_B.pkl'
                    , "wb")
pickle.dump(subj_dfs, pickle_out)
pickle_out.close()

## Prep A_dic

In [73]:
A_dic = {'train': {}, 'test':{}}

for tt in ['train', 'test']:
    for pn in ['positive', 'negative']:
        df = test_dic[tt][pn]
        A_dic[tt][pn] = df[df['asin'].isin(subj_dfs[tt][pn]['asin'])]

In [74]:
total = 0
old_total = 0
for tt in ['train', 'test']:
    for pn in ['positive', 'negative']:
        print('\n {} {}'.format(tt,pn))
        tot = A_dic[tt][pn].shape[0]
        new = subj_dfs[tt][pn].shape[0]
        dropped = tot - new
        print('Dropped: {0} ({1:.1%})'.format(dropped, dropped/tot))
        total += new
        old_total += tot
print('\nOld total:', old_total)
print('New total:', total)
print('Removed a total of:', old_total - total, 'reviews')


 train positive
Dropped: 346 (6.0%)

 train negative
Dropped: 394 (6.9%)

 test positive
Dropped: 85 (6.0%)

 test negative
Dropped: 95 (6.8%)

Old total: 14274
New total: 13354
Removed a total of: 920 reviews


In [75]:
pickle_out = open(data_path
                    + 'reviews_wout_top_' + str(int(round(REMOVE_FRACTION * 100))) + 'pct_' + REMOVE + '_A.pkl'
                    , "wb")
pickle.dump(A_dic, pickle_out)
pickle_out.close()